In [70]:
# imports
import pandas as pd
import json
import os
# change directory
# os.chdir(os.path.dirname(os.getcwd()))

In [71]:
os.getcwd()

'c:\\Users\\sonny\\Documents\\Data Science\\TechLabs Project\\src'

In [72]:
# settings for Lottie (Animations)

data_points_url = 'https://assets5.lottiefiles.com/packages/lf20_6dboqita.json'
wondering_url = 'https://assets4.lottiefiles.com/packages/lf20_pcoatxlk.json'
typing_url = 'https://assets1.lottiefiles.com/temporary_files/r5WAZZ.json'
reportlist_url='https://assets9.lottiefiles.com/packages/lf20_dq6whaxi.json'
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

In [73]:
# set application title
application_name = 'BraSolar' 
report_status = ('Status updates will be displayed after starting the analysis.\nAnalysis at 18% - weather data loaded.\nAnalysis at 23%')

In [74]:
# import libraries & components

# import custom components
# in the future the source will need to be changed (add d07_visualisation in front of 'components')
from components import start_button, no_fig, your_fig

# 
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc 
from dash_extensions import Lottie
import pandas as pd
import os


In [75]:
# custom cardbody for status report
status_card = dbc.CardBody([
    html.H4('Status report'),
    html.P('Status updates will be displayed after starting the analysis.'),
    html.P('Analysis at 18% - weather data loaded.')

],id='status_card',style={'font_family':'Courier'})

In [76]:
# custom html.P wrapper to include status reports
status_p = html.P(['Status updates will be displayed after starting the analysis.'])

In [77]:
status_p.children = status_p.children+[html.Br(),'lololo']

In [78]:
status_p

P(['Status updates will be displayed after starting the analysis.', Br(None), 'lololo'])

In [79]:
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.title = application_name

app.layout = dbc.Container([
    # row one
    dbc.Row([
        # row one col one
        dbc.Col([
            # logo
            dbc.Card([
                dbc.CardImg(src='../references/solarlogo.png')
            ],className='mb-2'),
            # name and link to website
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink('Sönke Maibach',target='_blank',  # _blank opens a new tab on click
                    href='https://mabikono.de')
                ])
            ],style={'height':'16vh'}),
        ], width=2),
        # row one col two
        dbc.Col([
            # title and short info
            dbc.Card([
                dbc.CardBody([
                    html.H4('{}'.format(application_name)),
                    html.P('Find out if your chosen locations are suitable for solar power.')
                ])
            ], color = 'info',style={'height':'18vh'}),
        ], width=6),
        # row one col three
        # Dark/Light Mode and Language Dropdown
        dbc.Col([
            dbc.Card([
                dbc.Button('Turn on light',color='primary', id='theme_switch')
            ])
        ],width=2)
    ], className='m-2 mt-4 h-25'),
    # row two
    dbc.Row([
        # row two col one
        # description
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options,url=wondering_url, width='40%', height='40%')),
                dbc.CardBody([
                    html.H4('What is going on here?',className='card-title'),
                    html.P(
                        'Computing energy cost savings and ROI by installing '
                        'a solar system. Uses weather and topographical data '
                        'combined with personal consumption profile to find '
                        'location, tilt and alignment with maximum energy output, '
                        'energy cost savings or ROI.'
                    )
                ])
            ]),
        ], width=4),
        # row two col two
        # inputs and start button
        dbc.Col([
            dbc.Card([
                # card header for the inputs
                dbc.CardHeader(
                    Lottie(options=options,url=typing_url, height='25%', width='25%')
                ),
                dbc.CardBody([
                    html.H4('Tell me a bit about yourself'),
                    # input field for location
                    html.I("Choose a city, region or country."),
                    html.Br(),
                    dcc.Input(id="location_input", type="search",
                    value="Sachsen",
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    
                    
                    # radioitems resolution
                    html.I("Choose the resolution. A higher resolution means more data points (DP) will be analyzed."),
                    html.Br(),
                    dcc.RadioItems(id='resolution_radio',
                        options=[
                            {'label':'Fastest (~100 DP)','value':200},
                            {'label':'Balanced (~400 DP)','value':800},
                            {'label':'Very detailed (~1000 DP)','value':2000},
                        ],
                        value=200,
                        labelStyle={'display': 'block'}
                    ),

                    # Household Size input
                    html.I('How many people live in your household? Children count as 0.5.'),
                    html.Br(),
                    dcc.Input(id="household_input", type="number",
                    value=3,
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    # start button
                    start_button,

                    # choose output to plot
                    html.I('Choose what you want to see.'), html.Br(),
                    dcc.RadioItems(
                        options=[
                            {'label':'Maximum electricity output','value':'max_out'},
                            {'label':'Maximum energy cost savings','value':'max_savings'},
                            {'label':'Maximum ROI','value':'max_roi'},
                        ],
                        value='max_roi',
                        labelStyle={'display':'block'}
                    )

                ])
            ]),
        ], width=4),
        # row two col three
        # status updates
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(url=reportlist_url,options=options,height='20%',width='20%')),
                dbc.CardBody([
                    html.H4('Status report'),
                    html.P('Status updates will be displayed after starting the analysis.')
                ])
            ]),
        ], width=4),
    ],className='m-2 h-15'),
    # row three 
    dbc.Row([
        # row three col one
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='map_graph',figure={}),
                ])
            ]),
        ], width=6,style={}),
        # row three col two
        # additional graph next to map
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='histogram_graph',figure={}),
                ])
            ]),
        ], width=6),
    ],className='m-2',style={}),

    dbc.Row([
        # row four col one
        # graph to the left
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='ll_graph',figure={}),
                ])
            ]),
        ], width=3),
        # row three col two
        # centered graph
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='lm_graph',figure={}),
                ])
            ]),
        ], width=5),
        # row three col three
        # graph to the right
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='lr_graph',figure={}),
                ])
            ]),
        ], width=4),
        # hidden data storage
        dcc.Store(id='location_reference_store'),
        dcc.Store(id='weather_df_store'),
        dcc.Store(id='max_out_store'),
        dcc.Store(id='smallest_area_store')
    ],className='m-2 mb-4',style={}),
    

], fluid=True, style={'height':'100vh'})

In [80]:
# callback section
# connect dash components

#     return fig
@app.callback(
    Output('start_button','n_clicks'),
    Input('location_input','value'),
    Input('resolution_radio','value'),
    Input('household_input','value')
)
def reset_n_clicks(loc,res,hh):
    return 0


# button test callback
@app.callback(
    [# test output
        Output("example-output", "children"),
        # output map
        Output('map_graph','figure'),
        # right output
        Output('histogram_graph','figure'),
        # last row outputs
        Output('ll_graph','figure'),
        Output('lm_graph','figure'),
        Output('lr_graph','figure'),
    ],
    [Input("start_button", "n_clicks"),
    Input('location_input','value')]
)
def on_button_click(n,loc):
    if n ==0:
        return ["Not clicked."], no_fig, no_fig, no_fig, no_fig, no_fig
    else:
        # first status update
        status_p.children = ['Analysis started']

        # this is a fake path for testing purposes
        max_path ='../data/03_processed/max_out_'+loc.lower()+'.pqt'
        if os.path.exists(max_path):
            test_df = pd.read_parquet(max_path).reset_index()
        geo_path = '../references/polygons_'+loc.lower()+'.json'
        with open(geo_path) as gfile:
            jsonObject = json.load(gfile)
            gfile.close()

        # create map
        map_fig = px.choropleth_mapbox(test_df, geojson=jsonObject, locations='index', color='loss_temp',
                                color_continuous_scale="Viridis",
                                #    range_color=(0.1, .11),
                                
                                mapbox_style="carto-darkmatter",
                                zoom=6, center = {
                                    "lat": jsonObject['features'][int(100/2)]['bbox'][1],
                                    "lon": jsonObject['features'][int(100/2)]['bbox'][0]
                                },
                                opacity=0.8,
                                labels={'loss_temp':'loss by temp',
                                            'loss_soiling':'loss by dirt'},
                                hover_data={'loss_soiling':':.2f'
                                            #,'loss_by_temp':':.2f'
                                            },
                                height=600
                                )
        map_fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0}
        )


        # create histogram next to map
        hist_fig = px.histogram(test_df,x='savings_cost_ratio')


        # create scatter plot for ll_fig
        ll_fig = px.scatter(test_df.iloc[:10*n+40],x='loss_soiling',y='loss_temp')
        counter = [f"Clicked {n} times." + str(test_df.iloc[5,n])]
        
        return counter, map_fig, hist_fig, ll_fig, your_fig, your_fig

In [81]:
# run the server
# server can also be accessed via browser (http://127.0.0.1:8050/)
app.run_server(mode='inline')

G:\Anaconda\envs\mapenv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

